# AdventureWorks: resit questions

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app14-4")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@6fbe4f0e

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@795eddf3

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val cust_aw = hiveCxt.table("sqlzoo.CustomerAW")
val cust_addr = hiveCxt.table("sqlzoo.CustomerAddress")
val addr = hiveCxt.table("sqlzoo.Address")
val product = hiveCxt.table("sqlzoo.Product")
val order_det = hiveCxt.table("sqlzoo.SalesOrderDetail")
val order_head = hiveCxt.table("sqlzoo.SalesOrderHeader")
val prod_model = hiveCxt.table("sqlzoo.ProductModel")
val prod_model_prod = hiveCxt.table("sqlzoo.ProductModelProductDescription")
val prod_desc = hiveCxt.table("sqlzoo.ProductDescription")
val prod_cat = hiveCxt.table("sqlzoo.ProductCategory")

cust_aw: DataFrame = [customerid: int, namestyle: string ... 11 more fields]
cust_addr: DataFrame = [customerid: int, addressid: int ... 1 more field]
addr: DataFrame = [addressid: int, addressline1: string ... 5 more fields]
product: DataFrame = [productid: int, name: string ... 12 more fields]
order_det: DataFrame = [salesorderid: int, salesorderdetailid: int ... 4 more fields]
order_head: DataFrame = [salesorderid: int, revisionnumber: int ... 17 more fields]
prod_model: DataFrame = [productmodelid: int, name: string ... 1 more field]
prod_model_prod: DataFrame = [productmodelid: int, productdescriptionid: int ... 1 more field]
prod_desc: DataFrame = [productdescriptionid: int, description: string]
prod_cat: DataFrame = [productcategoryid: int, parentproductcategoryid: int ... 1 more field]

## 1.
**List the SalesOrderNumber for the customer "Good Toys" "Bike World"**

In [5]:
(order_head
 .join(cust_aw
       .filter(cust_aw("CompanyName").isin(List("Good Toys", "Bike World"): _*)),
       "CustomerID", joinType="right")
 .select("SalessOrderNumber", "CompanyName")
 .showHTML())

SalessOrderNumber,CompanyName
null,Bike World
SO71774,Good Toys


## 2.
**List the ProductName and the quantity of what was ordered by 'Futuristic Bikes'**

In [6]:
(product.join(order_det, "ProductID", joinType="right")
 .join(order_head, "SalesOrderID", joinType="right")
 .join(cust_aw.filter(cust_aw("CompanyName")==="Futuristic Bikes"), 
       "CustomerID", joinType="right")
 .select("Name", "OrderQty")
 .showHTML())

Name,OrderQty
ML Mountain Seat/Saddle,2
"Long-Sleeve Logo Jersey, L",2
"Classic Vest, S",3


## 3.
**List the name and addresses of companies containing the word 'Bike' (upper or lower case) and companies containing 'cycle' (upper or lower case). Ensure that the 'bike's are listed before the 'cycles's.**

In [7]:
(cust_aw.join(cust_addr, "CustomerID")
 .join(addr, "AddressID")
 .select("CompanyName", "AddressLine1", "AddressLine2", "City",
         "StateProvince", "CountryRegion", "PostalCode")
 .withColumn("flag", when(lower(col("CompanyName")).like("%bike%"), "bike")
             .when(lower(col("CompanyName")).like("%cycle%"), "cycle"))
 .na.drop(Seq("flag"))
 .orderBy("flag", "CompanyName")
 .showHTML())

CompanyName,AddressLine1,AddressLine2,City,StateProvince,CountryRegion,PostalCode,flag
A Bike Store,2251 Elliot Avenue,null,Seattle,Washington,United States,98104,bike
A Typical Bike Shop,"One Dancing, Rr",No. 25 Box 8033,Round Rock,Texas,United States,78664,bike
Advanced Bike Components,12345 Sterling Avenue,null,Irving,Texas,United States,75061,bike
Area Bike Accessories,6900 Sisk Road,null,Modesto,California,United States,95354,bike
Associated Bikes,5420 West 22500 South,null,Salt Lake City,Utah,United States,84101,bike
Authorized Bike Sales and Rental,490 Ne 4th St,null,Renton,Washington,United States,98055,bike
Basic Bike Company,15 East Main,null,Port Orchard,Washington,United States,98366,bike
Best o' Bikes,250880 Baur Blvd,null,Saint Louis,Missouri,United States,63103,bike
Big-Time Bike Store,9909 W. Ventura Boulevard,null,Camarillo,California,United States,93010,bike
Bike Dealers Association,9952 E. Lohman Ave.,null,Las Cruces,New Mexico,United States,88001,bike


## 4.
**Show the total order value for each CountryRegion. List by value with the highest first.**

In [8]:
(addr.join(order_head, (addr("AddressID")===order_head("ShipToAddressID")))
 .groupBy("CountryRegion")
 .sum("SubTotal")
 .withColumn("sum(SubTotal)", round($"sum(SubTotal)", 2))
 .orderBy(col("sum(SubTotal)").desc)
 .showHTML())

CountryRegion,sum(SubTotal)
United Kingdom,518096.42
United States,347336.69


## 5.
**Find the best customer in each region.**

In [9]:
(addr.join(order_head, (addr("AddressID")===order_head("ShipToAddressID")))
 .join(cust_aw, "CustomerID")
 .groupBy("CountryRegion", "CompanyName")
 .sum("SubTotal")
 .withColumn("sum(SubTotal)", round($"sum(SubTotal)", 2))
 .withColumn("sn", rank().over(
     Window.partitionBy("CountryRegion").orderBy(col("sum(SubTotal)").desc)))
 .filter(col("sn")===1)
 .select("CountryRegion", "CompanyName", "sum(SubTotal)")
 .showHTML())

CountryRegion,CompanyName,sum(SubTotal)
United Kingdom,Action Bicycle Specialists,108561.83
United States,Eastside Department Store,83858.43


In [10]:
spark.stop()